In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import os
import pickle

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

Instructions for updating:
non-resource variables are not supported in the long term
['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """load data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding = 'latin')
        return data['data'], data['labels']
    
class CifarData:
    """定义一个类处理cifar的数据"""
    def __init__(self, filenames, need_shuffle): #对于训练集来说需要shuffle，打乱数据以增加训练成效，对于测试集来说就不需要。
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename) 
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data) #纵向组合成一个矩阵
        self._data = self._data / 127.5 - 1   
        self._labels = np.hstack(all_labels) #横向组合成一个矩阵
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
                
    def _shuffle_data(self):
        #[0,1,2,3,4,5] ->[3,4,2,5,1.0] 在训练集上做
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data , batch_labels

    
    '''至此处理完了cifar数据'''


train_filemnames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_data_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filemnames,True)
test_data = CifarData(test_data_filenames,False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data)
print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)
[[-0.97647059 -0.97647059 -0.97647059 ... -0.24705882 -0.25490196
  -0.27058824]
 [ 0.14509804  0.24705882  0.2627451  ... -0.45098039 -0.2627451
  -0.12941176]
 [-0.40392157 -0.38823529 -0.36470588 ... -0.16862745  0.27843137
   0.49019608]
 ...
 [-0.56078431 -0.54509804 -0.56078431 ...  0.9372549   0.91372549
   0.9372549 ]
 [ 0.30196078  0.28627451  0.28627451 ...  1.          1.
   1.        ]
 [ 0.70980392  0.30980392  0.12941176 ...  0.73333333  0.74117647
   0.74901961]]
[1 9 3 7 2 2 7 8 0 9]


In [4]:
"""在tensorflow里，要先把图搭建起来，再往里面填数据。"""

#把残差连接块提取出来做成一个函数
def 
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None,3072])  
y = tf.placeholder(tf.int64, [None])  

x_image = tf.reshape(x, [-1, 3, 32, 32])
#size 32*32
x_image = tf.transpose(x_image, perm = [0, 2, 3, 1])  #perm是交换的顺序



y_ = tf.layers.dense(flatten,10)

loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
#y_ ->softmax
#y ->one_hot
#loss = ylogy_


"""我们还对准确率感兴趣，所以现在来计算准确率"""
#indice
predict = tf.argmax(y_,1)   #取每一行中最大的数

#[1,0,1,1,1,0,1]类似的
correct_prediction = tf.equal(predict,y) 

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64)) #准确率就是我们上面所说的这个值的平均值

'''定义一个梯度下降的方法'''
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) 
    #AdamOptimizer是梯度下降的一个变种，1e-3是一个初始化的generate，并且我们优化的是loss值


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [5]:
init = tf.global_variables_initializer() 
# 构建一个初始化函数，用于初始化数据
batch_size = 20
train_steps = 1000
test_steps = 100
'''构建一个session，用于执行计算图'''
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss:%4.5f, acc:%4.5f'\
                 %(i+1,loss_val,acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_data_filenames, False)
            all_test_acc_val =[]
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Steps: %d, acc:%4.5f' %(i+1, test_acc))
#通过调用session的run函数执行计算图。可以计算loss, accuracy, train_op。加了train_op就是在训练，没加就只是在测试。
#feed_dict是我们输入的数据，x是cifar10的图片数据，y是label数据


[Train] Step: 500, loss:1.80729, acc:0.20000
[Train] Step: 1000, loss:1.53160, acc:0.45000
